# Introduction

In this notebook, our goal is to construct the generators of the classical Lie algebras $SO(n)$, $SU(n)$ and $SL(n)$. 

$SO(n)$ is the orthogonal group generating rotations in $n$-dimensional space:
https://en.wikipedia.org/wiki/Orthogonal_group

$SU(n)$ is the special unitary group:

https://en.wikipedia.org/wiki/Special_unitary_group

https://en.wikipedia.org/wiki/Structure_constants

$SL(n)$ is the special linear group:
https://en.wikipedia.org/wiki/Special_linear_group

$SU(n)$ and $SO(n)$, also to smaller extent $SL(n)$, have numerous applications in physics. 

 1. $SO(3)$, being the generators of rotations in 3D space is perhaps the most familiar and most readily understood example. $SO(3)$ is a subgroup of $SO(3,1)$, which plays an central part in the theory of special relativity, since it is the symmetry of flat Minkowski spacetime. Relativistic transformations (time dilation and space contraction when moving near the speed of light) - more commonly known as Lorentz transformations (https://en.wikipedia.org/wiki/Lorentz_transformation) - are generated by $SO(3,1)$. Furthermore, all known particles in the universe such as scalars, fermionic particles and gauge bosons must form representations of $SO(3,1)$. 
    
    
 2. In high-energy physics, $SU(2)$ and $SU(3)$, respectively, are the gauge groups of the weak interaction (more commonly known in its unified version with $U(1)$ as electroweak theory https://en.wikipedia.org/wiki/Electroweak_interaction) and quantum chromodynamics (https://en.wikipedia.org/wiki/Quantum_chromodynamics). The standard model of particle physics (https://en.wikipedia.org/wiki/Standard_Model) is based on the product gauge group $U(1)\times SU(2)\times SU(3)$, which describes the 3 known forces: electromagnetism, strong and weak nuclear forces. 

In [1]:
import numpy

# Structure constants and algebra closure

Let $T_{i}$ be the generators of a Lie algebra G, where $i$ ranges from 1 to $dim(G)$. The Lie algebra G is characterized by its structure constants $f_{ijk}$ which appear in the commutation relations of any two generators of the algebra. The commutation relation between any two generators $T_{i}$ and $T_{j}$ is

$$[T_{i}, T_{j}] = \sum_k f_{ijk} T_{k}$$

The closure of a set of generators (i.e. whether this set of generators form a Lie algebra) can be verified by checking that the commutation relations $[T_{i}, T_{j}] = \sum_k f_{ijk} T_{k}$ are satisfied for all generators.



In [2]:
def commutator(T_a, T_b):
    return (T_a @ T_b - T_b @ T_a)

Given a set of $n\times n$ matrices, we would like to check if this set forms an algebra. The way to do this without knowing the structure constants $f_{ijk}$ would be to check if the commutators of any two random matrices have the same form (up to some multiplicative constant) as any existing matrix in the set. This is done in the function "check_algebra_closure_no_f" below. To do this, the function $\texttt{numpy.linalg.lstsq(a,b)}$ is used. 
This function finds the approximate solution for the vector $\vec x$ in the linear matrix equation

$$A x = B$$

In this scenario, $x$ is the vector of length $dim(G)$ of nonvanishing structure constants, 

$$x = \left(f_{i_1,j_1,k_1}, \ldots, f_{i_n, j_n, k_n} \right)$$

and $B$ is the commutator of any two matrices in the set, 

$$B = [T_i, T_j]$$ reshaped/flattened as a vector of length $n^2$. 

$A$ is now a matrix of dimensions $n^2\times dim(G)$ comprising of all the matrices in the set (flattened to a vector)

$$A = [T^{flattened}_1, T^{flattened}_2, \ldots, T^{flattened}_{dim(G)}]$$

For each pair of generators $(T_i, T_j)$, this procedure is repeated. 

In [3]:
def check_algebra_closure_no_f(gens):   
    dim = gens.shape[0]
    nm = gens.shape[1]
    residue = numpy.zeros((dim,dim), dtype = numpy.complex128)  
    residue_list = []
    
    for i in range(dim):
        for j in range(dim):
            residue[i,j] = numpy.linalg.lstsq(gens.reshape(dim,nm*nm).T, commutator(gens[i],gens[j]).ravel(), rcond = None)[1]
            if numpy.allclose(0, residue[i,j])==False:
                residue_list.append(((i,j),residue[i,j]))
                
    if len(residue_list) !=0:
        return 'This set of matrices is not an algebra', residue_list
    else:
        return True

Once we have established that the set of matrices of interest forms a Lie algebra, we can proceed to calculate its structure constants $f_{ijk}$:

$$f_{ijk} = tr(T_{i}[T_{j},T_{k}])$$

and its Cartan-Killing metric that can be used to lower indices (the inverse metric raises the indices)

$$K_{ij} = (T_{i})_N^M (T_{j})_M^N$$

so that

$$f_{ij}^k = K^{-1}_{kp} f_{ijp}$$

In [40]:
def compute_f_ijk(gens):
    dim = gens.shape[0]
    n = gens.shape[1]
    cc = numpy.zeros((dim,dim,n,n),dtype = numpy.complex128)
    #Cartan-Killing metric
    k_ij = numpy.einsum('iNM,jMN->ij', gens, gens, optimize='greedy')
    #Commutators
    for i in range(dim):
        for j in range(dim):
            cc[i,j] = commutator(gens[i], gens[j])
            
    f_ijk = numpy.einsum('ijMN, kNM -> ijk', cc, gens, optimize='greedy')
    f_ij_K = numpy.einsum('ijk, kK-> ijK', f_ijk, numpy.linalg.inv(k_ij), optimize='greedy')
    
    #Extract unique values of f_ij^k
    f_list = []
    for i in range(dim):
        for j in range(dim):
            for k in range(dim):
                if numpy.round(f_ij_K[i,j,k],4) != 0. and i<j<k:
                        f_list.append(([i,j,k], numpy.round(f_ij_K[i,j,k],3)))
    return k_ij, len(f_list), f_list, f_ij_K

In [9]:
def check_algebra_closure_with_f(gens):
    dim = gens.shape[0]
    n = gens.shape[1]
    f_ijK = compute_f_ijk(gens)[3]
    cc = numpy.zeros((dim,dim,n,n),dtype = numpy.complex128)
    f_T_k = numpy.zeros((dim,dim,n,n),dtype = numpy.complex128)
    truth_list = []
    
    for i in range(dim):
        for j in range(dim):
            cc[i,j] = commutator(gens[i], gens[j])
            f_T_k[i,j] = numpy.einsum('k, kmn -> mn', f_ijK[i,j], gens, optimize='greedy')
            if numpy.allclose(cc[i,j],f_T_k[i,j]) == False:
                truth_list.append((i,j))
    
    if len(truth_list) !=0:
        return False, truth_list
    else:
        return True
    
           
   

# SO(n) generators

$SO(n)$ has $n(n-1)/2$ generators that are real, antisymmetric matrices. Each matrix is specified by a pair of entries $(i,j)=1$ and  $(j,i) = -1$. 

In [6]:
def generate_SO(n):
    dim = int(n*(n-1)/2)
    gens = numpy.zeros((dim,n,n), dtype = numpy.complex128)
    ij_pair = [(i,j) for i in range(n) for j in range(n) if i<j]
    for a, (i,j) in enumerate(ij_pair):
        gens[a,i,j] = 1.
        gens[a,j,i] =-1.
    return gens

In [7]:
#Check for n = 3: SO(3)
so3 = generate_SO(3)
so3

array([[[ 0.+0.j,  1.+0.j,  0.+0.j],
        [-1.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j]],

       [[ 0.+0.j,  0.+0.j,  1.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j],
        [-1.+0.j,  0.+0.j,  0.+0.j]],

       [[ 0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  1.+0.j],
        [ 0.+0.j, -1.+0.j,  0.+0.j]]])

In [10]:
check_algebra_closure_no_f(generate_SO(3)), check_algebra_closure_with_f(so3)

(True, True)

In [12]:
#For SO(3), there is only 1 unique structure constant:
compute_f_ijk(so3)[1:3]

(1, [([0, 1, 2], (-1+0j))])

In [46]:
#Deliberately modify one of the generators to test that the algebra closure is False:
so3_false = numpy.array([[[ 0.+0.j,  1.+0.j,  0.+0.j],
        [+4.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j]],

       [[ 0.+0.j,  0.+0.j,  1.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j],
        [-1.+0.j,  0.+0.j,  0.+0.j]],

       [[ 0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  1.+0.j],
        [ 0.+0.j, -1.+0.j,  0.+0.j]]])
                  
check_algebra_closure_no_f(so3_false)

('This set of matrices is not an algebra',
 [((0, 1), (12.5+0j)),
  ((0, 2), (12.5+0j)),
  ((1, 0), (12.5+0j)),
  ((1, 2), (1.4705882352941175+0j)),
  ((2, 0), (12.5+0j)),
  ((2, 1), (1.4705882352941175+0j))])

In [15]:
so4 = generate_SO(4)
check_algebra_closure_no_f(so4), check_algebra_closure_with_f(so4)

(True, True)

In [17]:
print('The structure constants of SO(4) are:')
compute_f_ijk(so4)[1:3]

The structure constants of SO(4) are:


(4,
 [([0, 1, 3], (-1+0j)),
  ([0, 2, 4], (-1+0j)),
  ([1, 2, 5], (-1+0j)),
  ([3, 4, 5], (-1+0j))])

In [18]:
so6 = generate_SO(6)
check_algebra_closure_no_f(so6), check_algebra_closure_with_f(so6)

(True, True)

In [19]:
print('The structure constants of SO(6) are:')
compute_f_ijk(so6)[1:3]

The structure constants of SO(6) are:


(20,
 [([0, 1, 5], (-1+0j)),
  ([0, 2, 6], (-1+0j)),
  ([0, 3, 7], (-1+0j)),
  ([0, 4, 8], (-1+0j)),
  ([1, 2, 9], (-1+0j)),
  ([1, 3, 10], (-1+0j)),
  ([1, 4, 11], (-1+0j)),
  ([2, 3, 12], (-1+0j)),
  ([2, 4, 13], (-1+0j)),
  ([3, 4, 14], (-1+0j)),
  ([5, 6, 9], (-1+0j)),
  ([5, 7, 10], (-1+0j)),
  ([5, 8, 11], (-1+0j)),
  ([6, 7, 12], (-1+0j)),
  ([6, 8, 13], (-1+0j)),
  ([7, 8, 14], (-1+0j)),
  ([9, 10, 12], (-1+0j)),
  ([9, 11, 13], (-1+0j)),
  ([10, 11, 14], (-1+0j)),
  ([12, 13, 14], (-1+0j))])

# SU(n) generators

$SU(n)$ has $n^2-1$ generators, of which:
1. $n(n-1)/2$ are antisymmetric (with off-diagonal entries of $i$ and $-i$)
2. $n(n-1)/2$ are symmetric (with off-diagonal entries of 1)
3. $(n-1)$ are diagonal (traceless)

$SU(n)$ can be generated recursively from $SU(n-1)$ by adding the extra row and column and populate the entries 
to make sure that the above rules are obeyed. 

In the code below, we use SU(2) generators and build up recursively from there for $n\geq 3$.
Note that for the diagonal generators in this code, the normalization constant is chosen to be $\sqrt{\frac{2}{n(n-1)}}$. The numerical values of the structure constants involving these diagonal generators will change if a different normalization constant is used. 

## Construct SU(n) generators recursively

In [20]:
def generate_SU(n):
    def traceless_diag(n):
        tot_arr = numpy.zeros(n)
        for i in range(n-1):
            tot_arr[i] = -1
        tot_arr[n-1] = n-1
        return tot_arr
    #n<2
    if n==0 or n==1:
        return('Choose n>=2')
    #n=2
    if n==2:
        SU2_gens = numpy.zeros((3,2,2), dtype = numpy.complex128)
        SU2_gens[0] = numpy.array([[0.,1],[1,0]])
        SU2_gens[1] = numpy.array([[0.,-1j],[1j,0]])
        SU2_gens[2] = numpy.array([[1.,0],[0,-1]])
        return SU2_gens
    #n>2
    else:        
        dim = n**2-1
        dim_m_1 = (n-1)**2-1
        gens = numpy.zeros((dim,n,n), dtype = numpy.complex128)
        gens_m_1 = generate_SU(n-1)
        
        for i in range(dim_m_1):
            gens[i]= numpy.append(numpy.append(gens_m_1[i], numpy.zeros((n-1,1)), axis=1),
             numpy.zeros((1,n)), axis =0)
            
        #Those generators with 1 entries
        for a in range(dim_m_1, dim_m_1+(n-1)):
            gens[a,a%(n-1),n-1] = 1
            gens[a, n-1, a%(n-1)] = 1
    
        #Those generators with 1j entries
        for a in range(dim_m_1+(n-1), dim_m_1+2*(n-1)):
            gens[a,a%(n-1),n-1] = -1j
            gens[a, (n-1), a%(n-1)] = 1j
    
        #The generator with diagonal entries
        gens[dim-1] = (2**0.5/(n*(n-1))**0.5)*numpy.diag(traceless_diag(n))
        
        return gens

In [21]:
generate_SU(1)

'Choose n>=2'

In [22]:
su2 = generate_SU(2)
su2

array([[[ 0.+0.j,  1.+0.j],
        [ 1.+0.j,  0.+0.j]],

       [[ 0.+0.j, -0.-1.j],
        [ 0.+1.j,  0.+0.j]],

       [[ 1.+0.j,  0.+0.j],
        [ 0.+0.j, -1.+0.j]]])

In [23]:
check_algebra_closure_no_f(su2), check_algebra_closure_with_f(su2)

(True, True)

In [26]:
su3 = generate_SU(3)

check_algebra_closure_no_f(su3), check_algebra_closure_with_f(su3)

(True, True)

In [42]:
print('The structure constants of SU(3) are:')
compute_f_ijk(su3)[1:3]

The structure constants of SU(3) are:


(9,
 [([0, 1, 2], 2j),
  ([0, 3, 6], 1j),
  ([0, 4, 5], 1j),
  ([1, 3, 4], -1j),
  ([1, 5, 6], -1j),
  ([2, 3, 5], -1j),
  ([2, 4, 6], 1j),
  ([3, 5, 7], -1.732j),
  ([4, 6, 7], -1.732j)])

In [29]:
su4 = generate_SU(4)

check_algebra_closure_no_f(su4), check_algebra_closure_with_f(su4)

(True, True)

In [41]:
print('The structure constants of SU(4) are:')
compute_f_ijk(su4)[1:3]

The structure constants of SU(4) are:


(29,
 [([0, 1, 2], 2j),
  ([0, 3, 6], 1j),
  ([0, 4, 5], 1j),
  ([0, 9, 13], 1j),
  ([0, 10, 12], 1j),
  ([1, 3, 4], -1j),
  ([1, 5, 6], -1j),
  ([1, 9, 10], 1j),
  ([1, 12, 13], 1j),
  ([2, 3, 5], -1j),
  ([2, 4, 6], 1j),
  ([2, 9, 12], 1j),
  ([2, 10, 13], -1j),
  ([3, 5, 7], -1.732j),
  ([3, 8, 13], 1j),
  ([3, 10, 11], 1j),
  ([4, 6, 7], -1.732j),
  ([4, 8, 12], 1j),
  ([4, 9, 11], 1j),
  ([5, 8, 10], -1j),
  ([5, 11, 13], -1j),
  ([6, 8, 9], -1j),
  ([6, 11, 12], -1j),
  ([7, 8, 11], 1.155j),
  ([7, 9, 12], -0.577j),
  ([7, 10, 13], -0.577j),
  ([8, 11, 14], -1.633j),
  ([9, 12, 14], -1.633j),
  ([10, 13, 14], -1.633j)])

In [35]:
su5 = generate_SU(5)

check_algebra_closure_no_f(su5), check_algebra_closure_with_f(su5)

(True, True)

In [43]:
print('The structure constants of SU(5) are:')
compute_f_ijk(su5)[1:3]

The structure constants of SU(5) are:


(66,
 [([0, 1, 2], 2j),
  ([0, 3, 6], 1j),
  ([0, 4, 5], 1j),
  ([0, 9, 13], 1j),
  ([0, 10, 12], 1j),
  ([0, 16, 21], 1j),
  ([0, 17, 20], 1j),
  ([1, 3, 4], -1j),
  ([1, 5, 6], -1j),
  ([1, 9, 10], 1j),
  ([1, 12, 13], 1j),
  ([1, 16, 17], 1j),
  ([1, 20, 21], 1j),
  ([2, 3, 5], -1j),
  ([2, 4, 6], 1j),
  ([2, 9, 12], 1j),
  ([2, 10, 13], -1j),
  ([2, 16, 20], 1j),
  ([2, 17, 21], -1j),
  ([3, 5, 7], -1.732j),
  ([3, 8, 13], 1j),
  ([3, 10, 11], 1j),
  ([3, 17, 22], 1j),
  ([3, 18, 21], 1j),
  ([4, 6, 7], -1.732j),
  ([4, 8, 12], 1j),
  ([4, 9, 11], 1j),
  ([4, 16, 22], 1j),
  ([4, 18, 20], 1j),
  ([5, 8, 10], -1j),
  ([5, 11, 13], -1j),
  ([5, 17, 18], 1j),
  ([5, 21, 22], 1j),
  ([6, 8, 9], -1j),
  ([6, 11, 12], -1j),
  ([6, 16, 18], 1j),
  ([6, 20, 22], 1j),
  ([7, 8, 11], 1.155j),
  ([7, 9, 12], -0.577j),
  ([7, 10, 13], -0.577j),
  ([7, 16, 20], -0.577j),
  ([7, 17, 21], -0.577j),
  ([7, 18, 22], 1.155j),
  ([8, 11, 14], -1.633j),
  ([8, 15, 22], 1j),
  ([8, 18, 19], 1j),
  ([9,

# SL(n) generators from SU(n) generators

To convert $SU(n)$ generators to $SL(n)$ generators, do the following:

1. Multiply those generators with antisymmetric entries of $(-i, i)$ by $i$. This will create the generators
of the $SO(n)$ subgroup of $SL(n)$. 

2. Keep the rest of the generators as they are. 

In [31]:
def generate_SL_from_SU(n):
    gens = generate_SU(n)
    #The number of antisym matrices is n*(n-1)/2
    #gens_1j = numpy.zeros((n*(n-1)/2),n,n)
    #Pick out those generators with 1j entries
    for i in range(len(gens)):
        if numpy.iscomplex(gens[i]).any() == True:
            gens[i] = gens[i]*1j
    return gens

In [32]:
sl3 = generate_SL_from_SU(3)
check_algebra_closure_no_f(sl3), check_algebra_closure_with_f(sl3)

(True, True)

In [44]:
print('The structure constants of SL(3) are:')
compute_f_ijk(sl3)[1:3]

The structure constants of SL(3) are:


(9,
 [([0, 1, 2], (-2+0j)),
  ([0, 3, 6], (1+0j)),
  ([0, 4, 5], (1+0j)),
  ([1, 3, 4], (1+0j)),
  ([1, 5, 6], (1+0j)),
  ([2, 3, 5], (-1+0j)),
  ([2, 4, 6], (1+0j)),
  ([3, 5, 7], (1.732+0j)),
  ([4, 6, 7], (1.732+0j))])

In [45]:
#For comparison to those of SU(3)
print('The structure constants of SU(3) are:')
compute_f_ijk(su3)[1:3]

The structure constants of SU(3) are:


(9,
 [([0, 1, 2], 2j),
  ([0, 3, 6], 1j),
  ([0, 4, 5], 1j),
  ([1, 3, 4], -1j),
  ([1, 5, 6], -1j),
  ([2, 3, 5], -1j),
  ([2, 4, 6], 1j),
  ([3, 5, 7], -1.732j),
  ([4, 6, 7], -1.732j)])